In [9]:
from tokenizers import Tokenizer
from transformers import PreTrainedTokenizerFast
from tokenizers.models import WordLevel, BPE
from tokenizers.pre_tokenizers import Whitespace,Split,ByteLevel
from tokenizers.normalizers import Lowercase, NFKC
import os
import polars as pl
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
from tqdm import tqdm
import time
import json
from tokenizers.trainers import BpeTrainer, UnigramTrainer, WordLevelTrainer, WordPieceTrainer
import gc

from rdkit import Chem
from rdkit.Chem import AllChem
import atomInSmiles
import mapply
from collections import Counter
import pandas as pd

multiprocessing.cpu_count()

80

In [2]:
train_df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/train_v2.csv').select(
        pl.col('molecule'),
#         pl.col('bb1', 'bb2', 'bb3').cast(pl.UInt16),
        # pl.col('BRD4', 'HSA', 'sEH').cast(pl.UInt8),
    ).collect()
print(train_df.shape, train_df.estimated_size('gb'), 'GB')
train_df

(98415610, 1) 6.842148938216269 GB


molecule
str
"""C#CCOc1ccc(CNc…"
"""C#CCOc1ccc(CNc…"
"""C#CCOc1ccc(CNc…"
"""C#CCOc1ccc(CNc…"
"""C#CCOc1ccc(CNc…"
…
"""[N-]=[N+]=NCCC…"
"""[N-]=[N+]=NCCC…"
"""[N-]=[N+]=NCCC…"


In [3]:
test_df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/test_v4.csv').select(
        pl.col('molecule', 'mol_group'),
#         pl.col('bb1', 'bb2', 'bb3').cast(pl.UInt16),
        # pl.col('BRD4', 'HSA', 'sEH').cast(pl.UInt8),
    ).collect()
print(test_df.shape, test_df.estimated_size('gb'), 'GB')
test_df

(878022, 2) 0.06782370340079069 GB


molecule,mol_group
str,i64
"""C#CCCC[C@H](Nc…",2
"""C#CCCC[C@H](Nc…",2
"""C#CCCC[C@H](Nc…",2
"""C#CCCC[C@H](Nc…",2
"""C#CCCC[C@H](Nc…",2
…,…
"""Cn1ncc2cc(Nc3n…",0
"""[N-]=[N+]=NCCC…",0
"""COC(=O)c1ccnc(…",0


In [4]:
def replace_dy(smiles):
    #Convert your SMILES to a mol object.
    mol = Chem.MolFromSmiles(smiles)
    
    #Create a mol object to replace the Dy atom with.
    new_attachment = Chem.MolFromSmiles('C')
    
    #Get the pattern for the Dy atom
    dy_pattern = Chem.MolFromSmiles('[Dy]')
    
    #This returns a tuple of all possible replacements, but we know there will only be one.
    new_mol = AllChem.ReplaceSubstructs(mol, dy_pattern, new_attachment)[0]
    
    #Good idea to clean it up
    Chem.SanitizeMol(new_mol)
    
    # #Since you want 3D mols later, I'd suggest adding hydrogens. Note: this takes up a lot more memory for the obj.
    # Chem.AddHs(new_mol)
    return Chem.MolToSmiles(new_mol)

In [13]:
for i in range(100):
    smiles = train_df[i, 'molecule']
    new_smiles = replace_dy(smiles)
    print(smiles, new_smiles, '', sep = '\n')

C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1
C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1

C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1
C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1

C#CCOc1ccc(CNc2nc(NCc3ccc(OCC#C)cc3)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1
C#CCOc1ccc(CNc2nc(NCc3ccc(OCC#C)cc3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1

C#CCOc1ccc(CNc2nc(NCCNC(=O)C(=C)C)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1
C#CCOc1ccc(CNc2nc(NCCNC(=O)C(=C)C)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1

C#CCOc1ccc(CNc2nc(NCC(=O)NCC=C)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1
C#CCOc1ccc(CNc2nc(NCC(=O)NCC=C)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1

C#CCOc1ccc(CNc2nc(NCC(C)OCC=C)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1
C#CCOc1ccc(CNc2nc(NCC(C)OCC=C)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1

C#CCOc1ccc(CNc2nc(NCCCOCC=C)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1
C#CCOc1ccc(CNc2nc(NCCCOCC=C)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1

C#CCOc1ccc(CNc2nc(NCCOCC=C)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1
C#CCOc

In [9]:
%%time

for i in range(1000):
    smiles = train_df[i, 'molecule']
    ais = atomInSmiles.encode(smiles)
    ais_tokens = ais.split()
    print(i, smiles, '\n-->', ais_tokens)

0 C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1 
--> ['[CH;!R;C]', '#', '[C;!R;CC]', '[CH2;!R;CO]', '[O;!R;CC]', '[c;R;CCO]', '1', '[cH;R;CC]', '[cH;R;CC]', '[c;R;CCC]', '(', '[CH2;!R;CN]', '[NH;!R;CC]', '[c;R;NNN]', '2', '[n;R;CC]', '[c;R;NNN]', '(', '[NH;!R;CC]', '[CH2;!R;CN]', '[CH;R;CCN]', '3', '[CH2;R;CC]', '[CH2;R;CC]', '[CH2;R;CN]', '[N;R;CCC]', '3', '[c;R;CNN]', '3', '[cH;R;CC]', '[cH;R;CC]', '[cH;R;CN]', '[n;R;CN]', '[n;R;CN]', '3', ')', '[n;R;CC]', '[c;R;NNN]', '(', '[NH;!R;CC]', '[[C@@H];!R;CCN]', '(', '[CH2;!R;CC]', '[C;!R;CC]', '#', '[CH;!R;C]', ')', '[CH2;!R;CC]', '[C;!R;CNO]', '(', '=', '[O;!R;C]', ')', '[NH;!R;CDy]', '[Dy;!R;N]', ')', '[n;R;CC]', '2', ')', '[cH;R;CC]', '[cH;R;CC]', '1']
1 C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1 
--> ['[CH;!R;C]', '#', '[C;!R;CC]', '[CH2;!R;CO]', '[O;!R;CC]', '[c;R;CCO]', '1', '[cH;R;CC]', '[cH;R;CC]', '[c;R;CCC]', '(', '[CH2;!R;CN]', '[NH;!R;CC]', '[c;R;NNN]', '2', '[n;R;CC]', '[c;R;N

In [11]:
mapply.init(
    n_workers=-1,
    progressbar=True,
    chunk_size = 1024,
    max_chunks_per_worker = 32,
)

In [7]:
random_test_smiles = [Chem.MolToSmiles(Chem.MolFromSmiles(smiles), doRandom=True) for smiles in tqdm(test_df['molecule'])]
len(random_test_smiles)

100%|████████████████████████████████| 878022/878022 [08:11<00:00, 1785.21it/s]


878022

In [13]:
%%time
test_aiss = pd.Series(random_test_smiles).mapply(atomInSmiles.encode)

  0%|                                                  | 0/857 [00:00<?, ?it/s]

CPU times: user 8.39 s, sys: 1min 34s, total: 1min 42s
Wall time: 2min 4s


In [14]:
len(test_aiss)

878022

In [15]:
test_counter = Counter()

for i, ais in tqdm(enumerate(test_aiss), total = len(test_aiss)):
    test_counter.update(ais.split())
    if i % 100_000 == 0:
        print(len(test_counter))

  1%|▍                               | 11849/878022 [00:00<00:14, 60113.76it/s]

22


 13%|███▉                           | 112148/878022 [00:01<00:11, 67591.68it/s]

210


 24%|███████▌                       | 213858/878022 [00:03<00:09, 69418.22it/s]

...

 81%|█████████████████████████      | 709339/878022 [00:11<00:02, 72273.08it/s]

218


 92%|████████████████████████████▌  | 808480/878022 [00:12<00:00, 73264.31it/s]

219


100%|███████████████████████████████| 878022/878022 [00:13<00:00, 64014.16it/s]


In [18]:
vocab_tokens = set(test_counter.keys())
vocab_tokens

{'#',
 '(',
 ')',
 '-',
 '/',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '=',
 '[B;R;COO]',
 '[Br;!R;C]',
 '[C;!R;CCCC]',
 '[C;!R;CCCN]',
 '[C;!R;CCCO]',
 '[C;!R;CCC]',
 '[C;!R;CCCl]',
 '[C;!R;CCFF]',
 '[C;!R;CCOO]',
 '[C;!R;CCO]',
 '[C;!R;CC]',
 '[C;!R;CClCl]',
 '[C;!R;CFFF]',
 '[C;!R;CNN]',
 '[C;!R;CNO]',
 '[C;!R;CNS]',
 '[C;!R;CN]',
 '[C;!R;COO]',
 '[C;!R;COS]',
 '[C;!R;CSi]',
 '[C;!R;FFFO]',
 '[C;!R;NNO]',
 '[C;!R;NOO]',
 '[C;R;CCCC]',
 '[C;R;CCCF]',
 '[C;R;CCCN]',
 '[C;R;CCCO]',
 '[C;R;CCCS]',
 '[C;R;CCC]',
 '[C;R;CCFF]',
 '[C;R;CCN]',
 '[C;R;CCOO]',
 '[C;R;CCO]',
 '[C;R;CCS]',
 '[C;R;CNN]',
 '[C;R;CNO]',
 '[C;R;COO]',
 '[C;R;FFOO]',
 '[C;R;NNO]',
 '[C;R;NNS]',
 '[C;R;NOO]',
 '[C;R;NOS]',
 '[CH2;!R;CC]',
 '[CH2;!R;CF]',
 '[CH2;!R;CN]',
 '[CH2;!R;CO]',
 '[CH2;!R;CS]',
 '[CH2;!R;C]',
 '[CH2;!R;NN]',
 '[CH2;!R;NO]',
 '[CH2;!R;OO]',
 '[CH2;R;CC]',
 '[CH2;R;CN]',
 '[CH2;R;CO]',
 '[CH2;R;CS]',
 '[CH2;R;NS]',
 '[CH2;R;OO]',
 '[CH3;!R;C]',
 '[CH3;!R;N]',
 '[CH3;!R;O]',
 '[CH3;

In [19]:
canonical_vocab_tokens = set({'(': 6532149,
         ')': 6532149,
         '[cH;R;CC]': 5568170,
         '[c;R;CCC]': 2554097,
         '1': 2478596,
         '2': 2464676,
         '[n;R;CC]': 2135243,
         '3': 2097610,
         '=': 2014722,
         '[O;!R;C]': 1687485,
         '[CH2;R;CC]': 1663878,
         '[C;!R;CNO]': 1470627,
         '[c;R;CCN]': 1323266,
         '[NH;!R;CC]': 1311475,
         '[CH2;R;CN]': 1299482,
         '[c;R;NNN]': 1212854,
         '4': 1061140,
         '[cH;R;CN]': 1010361,
         '[NH;!R;CDy]': 878022,
         '[Dy;!R;N]': 878022,
         '[CH3;!R;C]': 850528,
         '[F;!R;C]': 799229,
         '[N;R;CCC]': 737228,
         '-': 600945,
         '[CH2;!R;CN]': 572601,
         '[CH2;!R;CC]': 492951,
         '[c;R;CNN]': 458964,
         '[c;R;CCO]': 449185,
         '[O;!R;CC]': 443575,
         '5': 363928,
         '[CH2;R;CO]': 323436,
         '[n;R;CN]': 321708,
         '[c;R;CCF]': 292119,
         '[CH;R;CCC]': 261400,
         '[O;R;CC]': 254098,
         '[CH3;!R;N]': 207532,
         '[CH3;!R;O]': 201802,
         '[c;R;CCS]': 195090,
         '[s;R;CC]': 183072,
         '[Cl;!R;C]': 175542,
         '[n;R;CCN]': 174961,
         '[[C@@H];R;CCN]': 172535,
         '[CH2;!R;CO]': 168762,
         '[O;!R;S]': 139358,
         '[n;R;CCC]': 136407,
         '[c;R;CNO]': 126874,
         '[[C@H];R;CCN]': 113896,
         '[cH;R;CS]': 113533,
         '[c;R;CCCl]': 112493,
         '[C;!R;CFFF]': 111745,
         '[C;R;CCCC]': 105490,
         '[[C@H];R;CCC]': 98497,
         '[CH;R;CC]': 91933,
         '[nH;R;CC]': 91057,
         '[cH;R;NN]': 90866,
         '[CH;R;CCO]': 86915,
         '#': 84656,
         '[OH;!R;C]': 81561,
         '[C;R;CCC]': 81394,
         '[o;R;CC]': 81077,
         '[[C@@H];R;CCC]': 78500,
         '[CH;R;CCN]': 76896,
         '[Br;!R;C]': 72767,
         '[C;!R;CN]': 70585,
         '[N;!R;C]': 70585,
         '6': 69530,
         '[nH;R;CN]': 69146,
         '[[C@@H];!R;CCN]': 68319,
         '[[C@H];!R;CCN]': 62844,
         '[c;R;BrCC]': 59614,
         '[N;!R;CCC]': 59087,
         '[C;R;CNO]': 58882,
         '[C;!R;COO]': 58542,
         '[c;R;NNS]': 54698,
         '[c;R;CClN]': 52348,
         '[cH;R;CO]': 48384,
         '[[O-];!R;N]': 44840,
         '[[N+];!R;COO]': 43742,
         '[O;!R;N]': 43742,
         '[CH;!R;CC]': 41479,
         '[S;!R;CNOO]': 41068,
         '[c;R;CNS]': 39988,
         '[CH;!R;CCN]': 39681,
         '[CH3;!R;S]': 38888,
         '[[C@@H];R;CCO]': 37049,
         '[NH;R;CC]': 34497,
         '[n;R;NN]': 33291,
         '[CH;!R;CCO]': 31578,
         '[C;R;CCCO]': 30152,
         '[NH;!R;CS]': 29984,
         '[[C@H];R;CCO]': 29908,
         '[CH;!R;CCC]': 29507,
         '[n;R;CO]': 29383,
         '[CH2;R;CS]': 28669,
         '[C;!R;CCCC]': 27457,
         '[c;R;NNO]': 25742,
         '[o;R;CN]': 25383,
         '[CH2;!R;CS]': 25369,
         '[C;R;CCCN]': 23378,
         '[C;!R;CCCO]': 21289,
         '[C;R;CCFF]': 21137,
         '[S;!R;CC]': 19887,
         '[CH2;!R;C]': 19883,
         '[[C@@];R;CCCC]': 17894,
         '[NH2;!R;C]': 16421,
         '[C;!R;CC]': 15367,
         '[S;!R;CCOO]': 15252,
         '[c;R;CCI]': 14210,
         '[I;!R;C]': 14210,
         '[[C@];R;CCCC]': 13460,
         '[C;!R;FFFO]': 13095,
         '[CH;!R;CFF]': 12112,
         '[CH;!R;C]': 11863,
         '[C;!R;CCCN]': 11208,
         '[C;R;CCN]': 10684,
         '[S;R;CC]': 10593,
         '[s;R;CN]': 9489,
         '[C;!R;CCO]': 8736,
         '[c;R;BrCN]': 8727,
         '[CH;!R;FFO]': 8575,
         '[N;!R;CCS]': 8228,
         '[cH;R;NO]': 8202,
         '[C;!R;CCFF]': 8130,
         '[CH2;R;OO]': 7977,
         '[C;R;CCO]': 7918,
         '[C;R;CCCF]': 7773,
         '[C;!R;CCC]': 7686,
         '[O;R;CN]': 7508,
         '[n;R;CS]': 7492,
         '[c;R;CFN]': 7295,
         '[C;!R;NNO]': 7153,
         '[N;R;CCS]': 7007,
         '[cH;R;NS]': 6937,
         '[C;!R;NOO]': 6810,
         '/': 6726,
         '[CH;!R;CCS]': 6347,
         '[N;R;CO]': 6135,
         '[S;R;CCOO]': 5696,
         '[S;!R;CCO]': 5484,
         '[C;R;NNO]': 5439,
         '[c;R;ClNN]': 5165,
         '7': 4554,
         '[C;R;CCOO]': 3893,
         '[n;R;CNN]': 3735,
         '[CH;R;CO]': 3691,
         '[c;R;BrCS]': 3671,
         '[n;R;NS]': 3381,
         '[c;R;NOS]': 3153,
         '[N;R;CC]': 3020,
         '[CH;R;CCF]': 2810,
         '[CH3;!R;Si]': 2736,
         '[NH2;!R;S]': 2716,
         '[[C@@H];R;CCF]': 2703,
         '[S;!R;NNOO]': 2577,
         '[c;R;NSS]': 2576,
         '[c;R;CClS]': 2558,
         '[CH;R;CNO]': 2386,
         '[CH;R;CCS]': 2255,
         '[c;R;COS]': 2218,
         '[C;R;CNN]': 2201,
         '[C;R;FFOO]': 2176,
         '[c;R;NOO]': 2160,
         '[[C@@H];!R;CCO]': 2098,
         '[nH;R;NN]': 2045,
         '[o;R;NN]': 2000,
         '[C;R;NOO]': 1948,
         '[[C@H];!R;CCO]': 1929,
         '[CH2;!R;CF]': 1824,
         '[CH;!R;CO]': 1798,
         '[N;R;CCN]': 1777,
         '[N;R;CN]': 1777,
         '[O;R;BC]': 1760,
         '[S;R;CNOO]': 1713,
         '[[C@H];R;CCF]': 1576,
         '[N;!R;CCO]': 1566,
         '[O;!R;CN]': 1566,
         '[N;!R;CN]': 1487,
         '[[N+];!R;NN]': 1487,
         '[[N-];!R;N]': 1487,
         '[[C@@];!R;CCCN]': 1461,
         '[S;!R;CS]': 1458,
         '[C;!R;CNN]': 1402,
         '[NH;!R;C]': 1402,
         '[N;R;CCO]': 1373,
         '[C;R;CCCS]': 1354,
         '[SH;!R;C]': 1320,
         '[CH;!R;FFN]': 1301,
         '[S;R;CCO]': 1262,
         '[[C@@];R;CCCN]': 1227,
         '[CH2;R;NS]': 1200,
         '[CH;R;CNN]': 1176,
         '[S;!R;C]': 1110,
         '[CH;!R;CCF]': 1106,
         '[CH2;!R;NN]': 1099,
         '[[n+];R;CCO]': 1098,
         '[CH;R;CN]': 1083,
         '[CH2;!R;NO]': 1049,
         '[C;!R;CSi]': 912,
         '[Si;!R;CCCC]': 912,
         '[c;R;COO]': 904,
         '[c;R;BCC]': 880,
         '[B;R;COO]': 880,
         '[C;R;NOS]': 864,
         '[C;R;CCS]': 864,
         '[CH;R;CSS]': 843,
         '[C;R;NNS]': 819,
         '[[C@];R;CCCN]': 783,
         '[C;!R;CCCl]': 778,
         '[CH;!R;BrC]': 755,
         '[C;!R;CCOO]': 746,
         '[C;!R;CClCl]': 743,
         '[C;!R;COS]': 736,
         '[CH2;!R;OO]': 721,
         '[c;R;ClNS]': 714,
         '[s;R;NN]': 692,
         '[C;R;COO]': 692,
         '[c;R;FNN]': 641,
         '[c;R;CSS]': 629,
         '[C;!R;CNS]': 629,
         '8': 76,
         '[[C@];!R;CCCN]': 5,
         '9': 4}.keys())
vocab_tokens == canonical_vocab_tokens

True

In [16]:
%%time
test_aiss = test_df['molecule'].to_pandas().mapply(atomInSmiles.encode)

100%|████████████████████████████████████████| 857/857 [01:21<00:00, 10.55it/s]


In [17]:
%%time
train_aiss = train_df['molecule'].to_pandas().mapply(atomInSmiles.encode)

100%|████████████████████████████████████| 1312/1312 [1:42:34<00:00,  4.69s/it]


CPU times: user 9min 1s, sys: 11min 6s, total: 20min 8s
Wall time: 1h 44min 21s


In [21]:
!mkdir /home/dangnh36/datasets/competitions/leash_belka/processed/features/ais

In [22]:
test_aiss_df = pl.DataFrame({
    'ais': test_aiss
})
test_aiss_df.write_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/features/ais/test.csv')

In [23]:
train_aiss_df = pl.DataFrame({
    'ais': train_aiss
})
train_aiss_df.write_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/features/ais/train.csv')

In [43]:
test_counter = Counter()

for i, ais in tqdm(enumerate(test_aiss), total = len(test_aiss)):
    test_counter.update(ais.split())
    if i % 100_000 == 0:
        print(len(test_counter))

  2%|▌                               | 16529/878022 [00:00<00:11, 75761.39it/s]

22


 13%|███▉                           | 111070/878022 [00:01<00:09, 79785.24it/s]

210


 25%|███████▌                       | 215754/878022 [00:02<00:08, 77906.98it/s]

...

 81%|█████████████████████████      | 709076/878022 [00:09<00:02, 64349.51it/s]

218


 93%|████████████████████████████▋  | 813494/878022 [00:10<00:00, 71372.13it/s]

219


100%|███████████████████████████████| 878022/878022 [00:11<00:00, 75557.04it/s]


In [30]:
counter = Counter()

for i, ais in tqdm(enumerate(train_aiss), total = len(train_aiss)):
    counter.update(ais.split())
    if i % 1_000_000 == 0:
        print(len(counter))

  0%|                              | 14310/98415610 [00:00<22:54, 71599.72it/s]

31


  1%|▎                           | 1009206/98415610 [00:12<21:04, 77027.50it/s]

196


  2%|▌                           | 2011917/98415610 [00:25<21:48, 73702.44it/s]

...

 99%|██████████████████████████▌| 97009497/98415610 [22:57<00:18, 75628.46it/s]

205


100%|██████████████████████████▉| 98010906/98415610 [23:10<00:06, 65012.92it/s]

205


100%|███████████████████████████| 98415610/98415610 [23:16<00:00, 70473.52it/s]


In [31]:
counter

Counter({'(': 724464228,
         ')': 724464228,
         '[cH;R;CC]': 557940715,
         '[n;R;CC]': 399809759,
         '[c;R;NNN]': 301983031,
         '2': 299117572,
         '[NH;!R;CC]': 293918352,
         '1': 269640426,
         '3': 256174720,
         '[c;R;CCC]': 189955444,
         '=': 184898262,
         '[c;R;CCN]': 171726338,
         '[O;!R;C]': 149345761,
         '[CH2;R;CC]': 140023654,
         '[CH2;!R;CN]': 120080014,
         '[C;!R;CNO]': 109895961,
         '[NH;!R;CDy]': 98415610,
         '[Dy;!R;N]': 98415610,
         '[cH;R;CN]': 94036837,
         '[CH3;!R;C]': 83331324,
         '[F;!R;C]': 81837675,
         '4': 76736360,
         '[CH2;!R;CC]': 73357132,
         '[CH2;R;CN]': 51502355,
         '[O;!R;CC]': 51189135,
         '[c;R;CNN]': 46817969,
         '[Cl;!R;C]': 46182354,
         '[c;R;CCF]': 39540214,
         '[c;R;CCO]': 38649511,
         '[CH3;!R;O]': 33336407,
         '[CH2;R;CO]': 32578891,
         '[N;R;CCC]': 32317315,
      

In [39]:
test_counter

Counter({'(': 6532149,
         ')': 6532149,
         '[cH;R;CC]': 5568170,
         '[c;R;CCC]': 2554097,
         '1': 2478596,
         '2': 2464676,
         '[n;R;CC]': 2135243,
         '3': 2097610,
         '=': 2014722,
         '[O;!R;C]': 1687485,
         '[CH2;R;CC]': 1663878,
         '[C;!R;CNO]': 1470627,
         '[c;R;CCN]': 1323266,
         '[NH;!R;CC]': 1311475,
         '[CH2;R;CN]': 1299482,
         '[c;R;NNN]': 1212854,
         '4': 1061140,
         '[cH;R;CN]': 1010361,
         '[NH;!R;CDy]': 878022,
         '[Dy;!R;N]': 878022,
         '[CH3;!R;C]': 850528,
         '[F;!R;C]': 799229,
         '[N;R;CCC]': 737228,
         '-': 600945,
         '[CH2;!R;CN]': 572601,
         '[CH2;!R;CC]': 492951,
         '[c;R;CNN]': 458964,
         '[c;R;CCO]': 449185,
         '[O;!R;CC]': 443575,
         '5': 363928,
         '[CH2;R;CO]': 323436,
         '[n;R;CN]': 321708,
         '[c;R;CCF]': 292119,
         '[CH;R;CCC]': 261400,
         '[O;R;CC]': 2540

In [44]:
len(counter), len(test_counter)

(208, 222)

In [32]:
for i in range(10):
    print(i, any([str(i) in k for k in counter.keys()]))

0 False
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True


In [10]:
with open('/home/dangnh36/datasets/competitions/leash_belka/processed/features/ais/train_counter.json', 'w') as f:
    json.dump(counter, f)

In [40]:
with open('/home/dangnh36/datasets/competitions/leash_belka/processed/features/ais/test_counter.json', 'w') as f:
    json.dump(test_counter, f)

In [37]:
[e for e in counter if 'dy' in e.lower()]

['[NH;!R;CDy]', '[Dy;!R;N]']

In [45]:
[e for e in test_counter if 'dy' in e.lower()]

['[NH;!R;CDy]', '[Dy;!R;N]']

In [41]:
len(set(counter.keys()).union(set(test_counter.keys())))

222

In [46]:
set(test_counter.keys()).difference(set(counter.keys()))

{'[C;!R;CNS]',
 '[C;R;CNN]',
 '[C;R;FFOO]',
 '[CH2;!R;NN]',
 '[CH2;!R;NO]',
 '[CH;!R;CCF]',
 '[CH;R;CCF]',
 '[CH;R;CN]',
 '[CH;R;CO]',
 '[S;!R;NNOO]',
 '[[C@@];R;CCCC]',
 '[[C@];R;CCCC]',
 '[c;R;CSS]',
 '[c;R;NOO]'}

In [50]:
set(counter.keys()).difference(set(test_counter.keys()))

set()

In [51]:
test_counter

Counter({'(': 6532149,
         ')': 6532149,
         '[cH;R;CC]': 5568170,
         '[c;R;CCC]': 2554097,
         '1': 2478596,
         '2': 2464676,
         '[n;R;CC]': 2135243,
         '3': 2097610,
         '=': 2014722,
         '[O;!R;C]': 1687485,
         '[CH2;R;CC]': 1663878,
         '[C;!R;CNO]': 1470627,
         '[c;R;CCN]': 1323266,
         '[NH;!R;CC]': 1311475,
         '[CH2;R;CN]': 1299482,
         '[c;R;NNN]': 1212854,
         '4': 1061140,
         '[cH;R;CN]': 1010361,
         '[NH;!R;CDy]': 878022,
         '[Dy;!R;N]': 878022,
         '[CH3;!R;C]': 850528,
         '[F;!R;C]': 799229,
         '[N;R;CCC]': 737228,
         '-': 600945,
         '[CH2;!R;CN]': 572601,
         '[CH2;!R;CC]': 492951,
         '[c;R;CNN]': 458964,
         '[c;R;CCO]': 449185,
         '[O;!R;CC]': 443575,
         '5': 363928,
         '[CH2;R;CO]': 323436,
         '[n;R;CN]': 321708,
         '[c;R;CCF]': 292119,
         '[CH;R;CCC]': 261400,
         '[O;R;CC]': 2540

In [57]:
token2ord = {}
for i, (token, count) in enumerate(test_counter.most_common()):
    print(token)
    token2ord[token] = (i, count)
token2ord

(
)
[cH;R;CC]
[c;R;CCC]
1
2
[n;R;CC]
3
=
[O;!R;C]
[CH2;R;CC]
[C;!R;CNO]
[c;R;CCN]
[NH;!R;CC]
[CH2;R;CN]
[c;R;NNN]
4
[cH;R;CN]
[NH;!R;CDy]
[Dy;!R;N]
[CH3;!R;C]
[F;!R;C]
[N;R;CCC]
-
[CH2;!R;CN]
[CH2;!R;CC]
[c;R;CNN]
[c;R;CCO]
[O;!R;CC]
5
[CH2;R;CO]
[n;R;CN]
[c;R;CCF]
[CH;R;CCC]
[O;R;CC]
[CH3;!R;N]
[CH3;!R;O]
[c;R;CCS]
[s;R;CC]
[Cl;!R;C]
[n;R;CCN]
[[C@@H];R;CCN]
[CH2;!R;CO]
[O;!R;S]
[n;R;CCC]
[c;R;CNO]
[[C@H];R;CCN]
[cH;R;CS]
[c;R;CCCl]
[C;!R;CFFF]
[C;R;CCCC]
[[C@H];R;CCC]
[CH;R;CC]
[nH;R;CC]
[cH;R;NN]
[CH;R;CCO]
#
[OH;!R;C]
[C;R;CCC]
[o;R;CC]
[[C@@H];R;CCC]
[CH;R;CCN]
[Br;!R;C]
[C;!R;CN]
[N;!R;C]
6
[nH;R;CN]
[[C@@H];!R;CCN]
[[C@H];!R;CCN]
[c;R;BrCC]
[N;!R;CCC]
[C;R;CNO]
[C;!R;COO]
[c;R;NNS]
[c;R;CClN]
[cH;R;CO]
[[O-];!R;N]
[[N+];!R;COO]
[O;!R;N]
[CH;!R;CC]
[S;!R;CNOO]
[c;R;CNS]
[CH;!R;CCN]
[CH3;!R;S]
[[C@@H];R;CCO]
[NH;R;CC]
[n;R;NN]
[CH;!R;CCO]
[C;R;CCCO]
[NH;!R;CS]
[[C@H];R;CCO]
[CH;!R;CCC]
[n;R;CO]
[CH2;R;CS]
[C;!R;CCCC]
[c;R;NNO]
[o;R;CN]
[CH2;!R;CS]
[C;R;CCCN]
[C;!R;CCCO]
[C;R;CCFF]

{'(': (0, 6532149),
 ')': (1, 6532149),
 '[cH;R;CC]': (2, 5568170),
 '[c;R;CCC]': (3, 2554097),
 '1': (4, 2478596),
 '2': (5, 2464676),
 '[n;R;CC]': (6, 2135243),
 '3': (7, 2097610),
 '=': (8, 2014722),
 '[O;!R;C]': (9, 1687485),
 '[CH2;R;CC]': (10, 1663878),
 '[C;!R;CNO]': (11, 1470627),
 '[c;R;CCN]': (12, 1323266),
 '[NH;!R;CC]': (13, 1311475),
 '[CH2;R;CN]': (14, 1299482),
 '[c;R;NNN]': (15, 1212854),
 '4': (16, 1061140),
 '[cH;R;CN]': (17, 1010361),
 '[NH;!R;CDy]': (18, 878022),
 '[Dy;!R;N]': (19, 878022),
 '[CH3;!R;C]': (20, 850528),
 '[F;!R;C]': (21, 799229),
 '[N;R;CCC]': (22, 737228),
 '-': (23, 600945),
 '[CH2;!R;CN]': (24, 572601),
 '[CH2;!R;CC]': (25, 492951),
 '[c;R;CNN]': (26, 458964),
 '[c;R;CCO]': (27, 449185),
 '[O;!R;CC]': (28, 443575),
 '5': (29, 363928),
 '[CH2;R;CO]': (30, 323436),
 '[n;R;CN]': (31, 321708),
 '[c;R;CCF]': (32, 292119),
 '[CH;R;CCC]': (33, 261400),
 '[O;R;CC]': (34, 254098),
 '[CH3;!R;N]': (35, 207532),
 '[CH3;!R;O]': (36, 201802),
 '[c;R;CCS]': (37,

In [58]:
for token in set(test_counter.keys()).difference(set(counter.keys())):
    print(token, token2ord[token])

[C;!R;CNS] (218, 629)
[C;R;CNN] (159, 2201)
[c;R;CSS] (217, 629)
[CH2;!R;NO] (196, 1049)
[S;!R;NNOO] (153, 2577)
[CH;R;CN] (195, 1083)
[[C@@];R;CCCC] (103, 17894)
[c;R;NOO] (161, 2160)
[CH;R;CCF] (149, 2810)
[CH2;!R;NN] (193, 1099)
[[C@];R;CCCC] (109, 13460)
[CH;R;CO] (144, 3691)
[C;R;FFOO] (160, 2176)
[CH;!R;CCF] (192, 1106)


In [3]:
with open('/home/dangnh36/datasets/competitions/leash_belka/processed/features/ais/train_counter.json', 'r') as f:
    train_counter = json.load(f)
    print(len(train_counter))

with open('/home/dangnh36/datasets/competitions/leash_belka/processed/features/ais/test_counter.json', 'r') as f:
    test_counter = json.load(f)
    print(len(test_counter))

208
222


In [4]:
new_tokens = list(set(test_counter.keys()).difference(set(train_counter.keys())))
len(new_tokens)

14

In [5]:
test_aiss = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/features/ais/test.csv').collect()
test_aiss

ais
str
"""[CH;!R;C] # [C…"
"""[CH;!R;C] # [C…"
"""[CH;!R;C] # [C…"
"""[CH;!R;C] # [C…"
"""[CH;!R;C] # [C…"
…
"""[CH3;!R;N] [n;…"
"""[[N-];!R;N] = …"
"""[CH3;!R;O] [O;…"


In [6]:
test_has_new_token = [any([new_token in ais for new_token in new_tokens]) for ais in test_aiss['ais']]
test_has_new_token

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 

In [7]:
test_df = test_df.with_columns(has_new_token = pl.Series(test_has_new_token))
test_df

molecule,mol_group,has_new_token
str,i64,bool
"""C#CCCC[C@H](Nc…",2,false
"""C#CCCC[C@H](Nc…",2,false
"""C#CCCC[C@H](Nc…",2,false
"""C#CCCC[C@H](Nc…",2,false
"""C#CCCC[C@H](Nc…",2,false
…,…,…
"""Cn1ncc2cc(Nc3n…",0,false
"""[N-]=[N+]=NCCC…",0,false
"""COC(=O)c1ccnc(…",0,false


In [8]:
test_df['has_new_token'].sum()

50048

In [9]:
50048 / 878022

0.05700084963702504

In [12]:
test_df.filter(pl.col('has_new_token') == True).group_by('mol_group').agg(pl.col('molecule').count()).sort('mol_group')

mol_group,molecule
i64,u32
1,48229
2,612
3,1207


**all new AIS tokens comes from new building blocks, where most of them come from non-triazine cores (mol_group = 1)**

In [13]:
test_counter = Counter(test_counter)
test_counter.most_common()

[('(', 6532149),
 (')', 6532149),
 ('[cH;R;CC]', 5568170),
 ('[c;R;CCC]', 2554097),
 ('1', 2478596),
 ('2', 2464676),
 ('[n;R;CC]', 2135243),
 ('3', 2097610),
 ('=', 2014722),
 ('[O;!R;C]', 1687485),
 ('[CH2;R;CC]', 1663878),
 ('[C;!R;CNO]', 1470627),
 ('[c;R;CCN]', 1323266),
 ('[NH;!R;CC]', 1311475),
 ('[CH2;R;CN]', 1299482),
 ('[c;R;NNN]', 1212854),
 ('4', 1061140),
 ('[cH;R;CN]', 1010361),
 ('[NH;!R;CDy]', 878022),
 ('[Dy;!R;N]', 878022),
 ('[CH3;!R;C]', 850528),
 ('[F;!R;C]', 799229),
 ('[N;R;CCC]', 737228),
 ('-', 600945),
 ('[CH2;!R;CN]', 572601),
 ('[CH2;!R;CC]', 492951),
 ('[c;R;CNN]', 458964),
 ('[c;R;CCO]', 449185),
 ('[O;!R;CC]', 443575),
 ('5', 363928),
 ('[CH2;R;CO]', 323436),
 ('[n;R;CN]', 321708),
 ('[c;R;CCF]', 292119),
 ('[CH;R;CCC]', 261400),
 ('[O;R;CC]', 254098),
 ('[CH3;!R;N]', 207532),
 ('[CH3;!R;O]', 201802),
 ('[c;R;CCS]', 195090),
 ('[s;R;CC]', 183072),
 ('[Cl;!R;C]', 175542),
 ('[n;R;CCN]', 174961),
 ('[[C@@H];R;CCN]', 172535),
 ('[CH2;!R;CO]', 168762),
 ('[O;

In [15]:
with open('/home/dangnh36/datasets/competitions/leash_belka/processed/features/ais/vocab.txt', 'w') as f:
    content = '\n'.join([e[0] for e in list(test_counter.most_common())])
    f.write(content)